# Train a Simple Audio Recognition Model

This notebook demonstrates how to train a 20 kB [Simple Audio Recognition](https://www.tensorflow.org/tutorials/sequences/audio_recognition) model to recognize keywords in speech.

The model created in this notebook is used in the [micro_speech](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech) example for [TensorFlow Lite for MicroControllers](https://www.tensorflow.org/lite/microcontrollers/overview).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/train/train_micro_speech_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/train/train_micro_speech_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


**Training is much faster using GPU acceleration.** Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and set **Hardware accelerator: GPU**. Training 15,000 iterations will take 1.5 - 2 hours on a GPU runtime.

## Configure Defaults

**MODIFY** the following constants for your specific use case.

In [21]:
# Delete log file
!rm -f jupyter_train_log.txt

In [2]:
# Redirect output to file
import sys
sys.stdout = open("jupyter_train_log.txt", "a")

In [3]:
# https://github.com/${owner}/${repo}/archive/${hash}.tar.gz
TF_URL = "https://github.com/tensorflow/tensorflow/archive/v1.15.5.tar.gz"

In [4]:
# Download speech commands dataset
if False:
    import os
    import wget
    import tarfile
    DATASET_DIR = "speech_dataset"
    SPEECH_DATASET_URL = "http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz"
    SPEECH_DATASET_ARCHIVE = "speech_commands_v0.02.tar.gz"
    SPEECH_DATASET_DIR = os.path.join(DATASET_DIR, "speech_commands_v0.02")

    if not os.path.exists(DATASET_DIR):
        os.mkdir(DATASET_DIR)
    if not os.path.exists(os.path.join(DATASET_DIR, SPEECH_DATASET_ARCHIVE)):
        wget.download(SPEECH_DATASET_URL, DATASET_DIR)
    if not os.path.exists(SPEECH_DATASET_DIR):
        tar_file = tarfile.open(os.path.join(DATASET_DIR, SPEECH_DATASET_ARCHIVE))
        tar_file.extractall(SPEECH_DATASET_DIR)
        tar_file.close()
# os.symlink(SPEECH_DATASET_DIR, "tf_train_speech_commands")

In [5]:
# Training files from https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/speech_commands
# copied to tf_train_speech_commands
SPEECH_EXAMPLE_PATH = "tf_train_speech_commands"
import sys
sys.path.append(SPEECH_EXAMPLE_PATH)

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All the other words will be used to train an "unknown" label and silent
# audio data with no spoken words will be used to train a "silence" label.
#WANTED_WORDS = "yes,no"
#WANTED_WORDS = "yes,no,up,down,left,right,on,off,stop,go"
#WANTED_WORDS = "yes,no,up,down,left,right,on,off"
WANTED_WORDS = "marvin"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=12000,3000 and LEARNING_RATE=0.001,0.0001
# will run 12,000 training loops in total, with a rate of 0.001 for the first
# 8,000, and 0.0001 for the final 3,000.
TRAINING_STEPS = "12000,3000"
#TRAINING_STEPS = "120,30"
LEARNING_RATE = "0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
TOTAL_STEPS = str(sum(map(lambda string: int(string), TRAINING_STEPS.split(","))))

# Print the configuration to confirm it
print("Training these words: %s" % WANTED_WORDS)
print("Training steps in each stage: %s" % TRAINING_STEPS)
print("Learning rate in each stage: %s" % LEARNING_RATE)
print("Total number of training steps: %s" % TOTAL_STEPS)

# How long each spectrogram timeslice is.
#WINDOW_SIZE_MS = 30 # default: 30
# How loud the background noise should be, between 0 and 1.
#TRAIN_BACKGROUND_VOLUME_RANGE = 0.1 # default: 0.1
# How many of the training samples have background noise mixed in.
#TRAIN_BACKGROUND_FREQUENCY = 0.8 # default: 0.8

**DO NOT MODIFY** the following constants as they include filepaths used in this notebook and data that is shared during training and inference.

In [6]:
# Calculate the percentage of 'silence' and 'unknown' training samples required
# to ensure that we have equal number of samples for each label.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants which are shared during training and inference
PREPROCESS = 'micro'
WINDOW_STRIDE = 20
MODEL_ARCHITECTURE = 'tiny_conv' # Other options include: single_fc, conv,
                      # low_latency_conv, low_latency_svdf, tiny_embedding_conv

# Constants used during training only
VERBOSITY = 'WARN'
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Constants for training directories and filepaths
DATASET_DIR =  'dataset/'
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # for training checkpoints and other files.

# Constants for inference directories and filepaths
import os
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
  os.mkdir(MODELS_DIR)
MODEL_TF = os.path.join(MODELS_DIR, 'model.pb')
MODEL_TFLITE = os.path.join(MODELS_DIR, 'model.tflite')
FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'float_model.tflite')
#MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'model.cc')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'micro_speech_model_data.cpp')
SAVED_MODEL = os.path.join(MODELS_DIR, 'saved_model')

QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN

## Setup Environment

Install Dependencies

In [7]:
#%tensorflow_version 1.x
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

**DELETE** any old data from previous runs


In [8]:
#!rm -rf {DATASET_DIR} {LOGS_DIR} {TRAIN_DIR} {MODELS_DIR}
!rm -rf {LOGS_DIR} {TRAIN_DIR} {MODELS_DIR}

Clone the TensorFlow Github Repository, which contains the relevant code required to run this tutorial.

In [32]:
#!git clone -q --depth 1 https://github.com/tensorflow/tensorflow
#!git clone --quiet --depth 1 --branch v1.15.5 https://github.com/tensorflow/tensorflow
#!rm -rf tensorflow
if not os.path.exists("tensorflow"):
    !git clone --quiet --depth 1 --branch v2.13.0 https://github.com/tensorflow/tensorflow

Load TensorBoard to visualize the accuracy and loss as training proceeds.


In [10]:
#%load_ext tensorboard
#%tensorboard --logdir {LOGS_DIR}

## Training

The following script downloads the dataset and begin training.

In [11]:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/speech_commands/train.py
# https://github.com/tensorflow/tensorflow/tree/r1.15/tensorflow/examples/speech_commands/train.py
#!python tensorflow/tensorflow/examples/speech_commands/train.py \
#!python ./tf_train_speech_commands/train.py \
#%run tf_train_speech_commands/train.py \
%run tensorflow/tensorflow/examples/speech_commands/train.py \
#--data_url=  \
--data_dir={DATASET_DIR} \
--wanted_words={WANTED_WORDS} \
--silence_percentage={SILENT_PERCENTAGE} \
--unknown_percentage={UNKNOWN_PERCENTAGE} \
--preprocess={PREPROCESS} \
--window_stride={WINDOW_STRIDE} \
--model_architecture={MODEL_ARCHITECTURE} \
--how_many_training_steps={TRAINING_STEPS} \
--learning_rate={LEARNING_RATE} \
--train_dir={TRAIN_DIR} \
--summaries_dir={LOGS_DIR} \
--verbosity={VERBOSITY} \
--eval_step_interval={EVAL_STEP_INTERVAL} \
--save_step_interval={SAVE_STEP_INTERVAL}
#--window_size_ms={WINDOW_SIZE_MS} \
#--background_volume={TRAIN_BACKGROUND_VOLUME_RANGE} \
#--background_frequency={TRAIN_BACKGROUND_FREQUENCY}
# added: window_size_ms background_volume background_frequency

2023-08-30 17:31:08.644630: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-08-30 17:31:08.667597: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3199920000 Hz
2023-08-30 17:31:08.671658: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56493b012e90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-08-30 17:31:08.671685: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
W0830 17:31:39.011559 140481279395648 deprecation.py:506] From /data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/tensorflow/tensorflow/examples/speech_commands/models.py:737: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. R

# Skipping the training

If you don't want to spend an hour or two training the model from scratch, you can download pretrained checkpoints by uncommenting the lines below (removing the '#'s at the start of each line) and running them.

In [12]:
#!curl -O "https://storage.googleapis.com/download.tensorflow.org/models/tflite/speech_micro_train_2020_05_10.tgz"
#!tar xzf speech_micro_train_2020_05_10.tgz

## Generate a TensorFlow Model for Inference

Combine relevant training results (graph, weights, etc) into a single file for inference. This process is known as freezing a model and the resulting model is known as a frozen model/graph, as it cannot be further re-trained after this process.

In [33]:
# Redirect output to file
!rm -f jupyter_freeze_log.txt
import sys
sys.stdout = open("jupyter_freeze_log.txt", "a")

In [34]:
tf.compat.v1.reset_default_graph()
!rm -rf {SAVED_MODEL}
#!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
#!python tf_train_speech_commands/freeze.py \
%run tensorflow/tensorflow/examples/speech_commands/freeze.py \
--wanted_words=$WANTED_WORDS \
--window_stride_ms=$WINDOW_STRIDE \
--preprocess=$PREPROCESS \
--model_architecture=$MODEL_ARCHITECTURE \
--start_checkpoint=$TRAIN_DIR$MODEL_ARCHITECTURE'.ckpt-'{TOTAL_STEPS} \
--save_format=saved_model \
--output_file={SAVED_MODEL}

W0831 13:40:41.424189 140481279395648 deprecation.py:323] From /data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/tensorflow/tensorflow/examples/speech_commands/freeze.py:183: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


## Generate a TensorFlow Lite Model

Convert the frozen graph into a TensorFlow Lite model, which is fully quantized for use with embedded devices.

The following cell will also print the model size, which will be under 20 kilobytes.

In [57]:
# Redirect output to file
!rm -f jupyter_model_log.txt
import sys
sys.stdout = open("jupyter_model_log.txt", "a")

In [58]:
import sys
# We add this path so we can import the speech processing modules.
#sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data
import models
import numpy as np

In [59]:
SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
#BACKGROUND_FREQUENCY = TRAIN_BACKGROUND_FREQUENCY
#BACKGROUND_VOLUME_RANGE = TRAIN_BACKGROUND_VOLUME_RANGE
TIME_SHIFT_MS = 100.0

#DATA_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
DATA_URL=''
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10

In [60]:
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))),
    SAMPLE_RATE, CLIP_DURATION_MS, WINDOW_SIZE_MS,
    WINDOW_STRIDE, FEATURE_BIN_COUNT, PREPROCESS)
audio_processor = input_data.AudioProcessor(
    DATA_URL, DATASET_DIR,
    SILENT_PERCENTAGE, UNKNOWN_PERCENTAGE,
    WANTED_WORDS.split(','), VALIDATION_PERCENTAGE,
    TESTING_PERCENTAGE, model_settings, LOGS_DIR)

In [62]:
# Tensorflow 1
#with tf.Session() as sess:
with tf.compat.v1.Session() as sess:
  float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
  float_tflite_model = float_converter.convert()
  float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
  print("Float model is %d bytes" % float_tflite_model_size)

  converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.inference_input_type = tf.lite.constants.INT8
  converter.inference_output_type = tf.lite.constants.INT8
  def representative_dataset_gen():
    for i in range(100):
      data, _ = audio_processor.get_data(1, i*1, model_settings,
                                         BACKGROUND_FREQUENCY, 
                                         BACKGROUND_VOLUME_RANGE,
                                         TIME_SHIFT_MS,
                                         'testing',
                                         sess)
      flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
      yield [flattened_data]
  converter.representative_dataset = representative_dataset_gen
  tflite_model = converter.convert()
  tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
  print("Quantized model is %d bytes" % tflite_model_size)


2023-08-31 13:48:23.834559: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2023-08-31 13:48:23.834662: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2023-08-31 13:48:23.838550: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:786] Optimization results for grappler item: graph_to_optimize
2023-08-31 13:48:23.838562: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   function_optimizer: function_optimizer did nothing. time = 0ms.
2023-08-31 13:48:23.838566: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   function_optimizer: function_optimizer did nothing. time = 0.001ms.
2023-08-31 13:48:23.847662: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2023-08-31 13:48:23.847976: I tensorflo

InvalidArgumentError: You must feed a value for placeholder tensor 'data/time_shift_offset' with dtype int32 and shape [2]
	 [[node data/time_shift_offset (defined at data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'data/time_shift_offset':
  File "data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/venv/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/venv/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/venv/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "data_fast/storage/cstollen/vedliot/projects/secure-microphone/training/venv/lib/python3.7/site-packages/tr

In [ ]:
# Tensorflow 2
if False:
    tf.compat.v1.disable_eager_execution()
    with tf.compat.v1.Session() as sess:
    # with tf.Session() as sess:
      float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
      float_tflite_model = float_converter.convert()
      float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
      print("Float model is %d bytes" % float_tflite_model_size)

      converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
      #converter.inference_input_type = tf.lite.constants.INT8
      #converter.inference_output_type = tf.lite.constants.INT8

      # TF 2
      converter.target_spec.supported_types = [tf.int8]

      def representative_dataset_gen():
        for i in range(100):
          data, _ = audio_processor.get_data(1, i*1, model_settings,
                                             BACKGROUND_FREQUENCY, 
                                             BACKGROUND_VOLUME_RANGE,
                                             TIME_SHIFT_MS,
                                             'testing',
                                             sess)
          flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
          yield [flattened_data]
      converter.representative_dataset = representative_dataset_gen
      tflite_model = converter.convert()
      tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
      print("Quantized model is %d bytes" % tflite_model_size)

## Testing the TensorFlow Lite model's accuracy

Verify that the model we've exported is still accurate, using the TF Lite Python API and our test set.

In [ ]:
# Tensorflow 1
# Helper function to run inference
def run_tflite_inference(tflite_model_path, model_type="Float"):
  # Load test data
  np.random.seed(0) # set random seed for reproducible test results.

  tf.compat.v1.disable_eager_execution()
  with tf.compat.v1.Session() as sess:
    
  # with tf.Session() as sess:
    test_data, test_labels = audio_processor.get_data(
        -1, 0, model_settings, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE,
        TIME_SHIFT_MS, 'testing', sess)
  test_data = np.expand_dims(test_data, axis=1).astype(np.float32)

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(tflite_model_path)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # For quantized models, manually quantize the input data from float to integer
  if model_type == "Quantized":
    input_scale, input_zero_point = input_details["quantization"]
    test_data = test_data / input_scale + input_zero_point
    test_data = test_data.astype(input_details["dtype"])

  correct_predictions = 0
  for i in range(len(test_data)):
    interpreter.set_tensor(input_details["index"], test_data[i])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    top_prediction = output.argmax()
    correct_predictions += (top_prediction == test_labels[i])

  print('%s model accuracy is %f%% (Number of test samples=%d)' % (
      model_type, (correct_predictions * 100) / len(test_data), len(test_data)))

In [ ]:
# Tensorflow 2
if False:
    # Helper function to run inference
    def run_tflite_inference(tflite_model_path, model_type="Float"):
      # Load test data
      np.random.seed(0) # set random seed for reproducible test results.
      with tf.Session() as sess:
        test_data, test_labels = audio_processor.get_data(
            -1, 0, model_settings, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE,
            TIME_SHIFT_MS, 'testing', sess)
      test_data = np.expand_dims(test_data, axis=1).astype(np.float32)

      # Initialize the interpreter
      interpreter = tf.lite.Interpreter(tflite_model_path)
      interpreter.allocate_tensors()

      input_details = interpreter.get_input_details()[0]
      output_details = interpreter.get_output_details()[0]

      # For quantized models, manually quantize the input data from float to integer
      if model_type == "Quantized":
        input_scale, input_zero_point = input_details["quantization"]
        test_data = test_data / input_scale + input_zero_point
        test_data = test_data.astype(input_details["dtype"])

      correct_predictions = 0
      for i in range(len(test_data)):
        interpreter.set_tensor(input_details["index"], test_data[i])
        interpreter.invoke()
        output = interpreter.get_tensor(output_details["index"])[0]
        top_prediction = output.argmax()
        correct_predictions += (top_prediction == test_labels[i])

      print('%s model accuracy is %f%% (Number of test samples=%d)' % (
          model_type, (correct_predictions * 100) / len(test_data), len(test_data)))

In [ ]:
# Compute float model accuracy
run_tflite_inference(FLOAT_MODEL_TFLITE)

# Compute quantized model accuracy
run_tflite_inference(MODEL_TFLITE, model_type='Quantized')

## Generate a TensorFlow Lite for MicroControllers Model
Convert the TensorFlow Lite model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers.

In [ ]:
# Install xxd if it is not available
#!apt-get update && apt-get -qq install xxd
# Convert to a C source file
#!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
#REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
#!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [ ]:
# Install xxd if it is not available
#!apt-get update && apt-get -qq install xxd
# Convert to a C source file
#!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
#MICRO_MODEL_FILENAME = "models/micro_speech_model_data.cpp"
#print(MODEL_TFLITE)
#print(MODEL_TFLITE_MICRO)
#print(MICRO_MODEL_FILENAME)
#!echo {MODEL_TFLITE} {MICRO_MODEL_FILENAME}
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
#print(REPLACE_TEXT)
!sed -i 's/_len/_size/g' {MODEL_TFLITE_MICRO}
!sed -i 's/unsigned/const unsigned/g' {MODEL_TFLITE_MICRO}
#!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}
!sed -i 's/'{REPLACE_TEXT}'/g_micro_speech_model_data/g' {MODEL_TFLITE_MICRO}
#!sed -i '1s/^/#include <cstdint>\n#include "micro_speech_model_data.h"\nconst unsigned int g_micro_speech_model_data_size = '{tflite_model_size}';\nalignas(16) const /g' {MODEL_TFLITE_MICRO}
!sed -i '1s/^/#include <cstdint>\n#include "micro_speech_model_data.h"\nalignas(16) /g' {MODEL_TFLITE_MICRO}

## Deploy to a Microcontroller

Follow the instructions in the [micro_speech](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech) README.md for [TensorFlow Lite for MicroControllers](https://www.tensorflow.org/lite/microcontrollers/overview) to deploy this model on a specific microcontroller.

**Reference Model:** If you have not modified this notebook, you can follow the instructions as is, to deploy the model. Refer to the [`micro_speech/train/models`](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/train/models) directory to access the models generated in this notebook.

**New Model:** If you have generated a new model to identify different words: (i) Update `kCategoryCount` and `kCategoryLabels` in [`micro_speech/micro_features/micro_model_settings.h`](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.h) and (ii) Update the values assigned to the variables defined in [`micro_speech/micro_features/model.cc`](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/micro_features/model.cc) with values displayed after running the following cell.

In [63]:
# Redirect output to file
#!rm -f micro_speech_model_data.cpp
#import sys
#sys.stdout = open("micro_speech_model_data.cpp", "w")# Print the C source file
#!cat {MODEL_TFLITE_MICRO}
#!cat {MICRO_MODEL_FILENAME}
#print("MODEL_TFLITE_MICRO: ", MODEL_TFLITE_MICRO)
#print("MICRO_MODEL_FILENAME: ", MICRO_MODEL_FILENAME)

In [ ]:
# Print log
#!cat jupyter_model_log.txt